In [1]:
import numpy as np
import cv2
from LOBOROBOT import LOBOROBOT # 载入机器人库
from gpiozero import Button
from gpiozero import LED
import time 
import threading # 线程W
import ctypes
import inspect
import time
import math

#超声模块依赖
from gpiozero import DistanceSensor
from time import sleep

# 载入显示库
import ipywidgets.widgets as widgets
from IPython.display import display
import libcamera
from picamera2 import Picamera2

# 载入yolo图像识别库和语言播报库
from ultralytics import YOLO
import gc
import pygame,time,os

pygame 2.1.2 (SDL 2.26.5, Python 3.11.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

#初始化两个摄像头
def get_camera():
    # 获取第一个摄像头
    picamera_0 = Picamera2(camera_num=0)
    config = picamera_0.create_preview_configuration(main={"format": 'RGB888', "size": (180, 120)},
                                               raw={"format": "SRGGB12", "size": (720, 480)})
    config["transform"] = libcamera.Transform(hflip=1, vflip=1)
    picamera_0.configure(config)
    picamera_0.start()
    
    # 获取第二个摄像头
    picamera_1 = Picamera2(camera_num=1)
    config = picamera_1.create_preview_configuration(main={"format": 'YUYV', "size": (1280, 720)},
                                                   raw={"format": "YUYV", "size": (1280, 720)})
    config["transform"] = libcamera.Transform(hflip=0, vflip=1)
    picamera_1.configure(config)
    picamera_1.start()
    return picamera_0,picamera_1

# 按键及LED初始化
Btn  = Button(19,pull_up=True)   # 按键端口
Gpin = LED(5)
Rpin = LED(6)

#SensorRight = Button(16,pull_up=True)     # 右侧红外避障传感器
#SensorLeft  = Button(12,pull_up=True)    # 左侧红外避障传感器

#初始化舵机
clbrobot = LOBOROBOT()  # 实例化机器人对象
clbrobot.t_stop(0)  # 停止

# Configure min and max servo pulse lengths
servo_min = 150  # Min pulse length out of 4096
servo_max = 600  # Max pulse length out of 4096

# 频率设置为50hz，适用于舵机系统。
clbrobot.set_servo_angle(10,85)  # 底座舵机 90 
clbrobot.set_servo_angle(9,60)  # 顶部舵机 145(9,50)
time.sleep(0.5)

# 按键标志位
keyflag = 0

# 按键控制函数
def keysacn():
    global keyflag
    print('*****************************************') 
    print('* makerobo Button Pressed!*') 
    print('*****************************************')
    Rpin.on()  # 打开红色LED
    Gpin.off() # 关闭绿色LED
    keyflag = 1   # 按键标志位置1

def released():
    print("button was released")
    Rpin.off()   # 关闭红色LED
    Gpin.on()    # 打开绿色LED

# 按键中断函数+
Btn.when_pressed = keysacn
Btn.when_released = released

In [3]:
# 云台摄像头可视化
image = widgets.Image(format='jpeg', width=180, height=120)
# image = widgets.Image(format='jpeg', width=1920, height=1080)
display(image)

Image(value=b'', format='jpeg', height='120', width='180')

In [4]:
# 云台摄像头可视化
image_thresh1_0 = widgets.Image(format='jpeg', width=180, height=120)
# image = widgets.Image(format='jpeg', width=1920, height=1080)
display(image_thresh1_0)

Image(value=b'', format='jpeg', height='120', width='180')

In [5]:
# 云台摄像头可视化
image_mask_0 = widgets.Image(format='jpeg', width=180, height=120)
# image = widgets.Image(format='jpeg', width=1920, height=1080)
display(image_mask_0)

Image(value=b'', format='jpeg', height='120', width='180')

In [6]:
# 识别摄像头可视化
image_1 = widgets.Image(format='jpeg', width=1280, height=720)
display(image_1)

Image(value=b'', format='jpeg', height='720', width='1280')

In [7]:
# 读取模型
#model = YOLO("best_ncnn_model")
# 超声测距停车阈值
dis_stop = 20
# 巡线算法 cx 阈值设置（进入新的场地需要重新校准）
cx_min = 70
cx_max = 120
# 直角弯转弯阈值
turn_angle_threshold = 20
# 语音模块延时(单位：s)（避免短时间内多次触发）
result_read_delay = 5
# 转弯操作延时阈值(单位：s)（避免短时间内多次触发）
turn_delay = 3


def Video_display():
    picamera_0, picamera_1 = get_camera()
    no_line_counter = 0  # 连续丢线帧计数器
    frame_counter = 0    # 抽帧计数
    detect_count = 0 # 识别次数
    # 初始化转弯参数
    _already_turn = False
    last_turn_time = 0   # 上一次转弯的时间戳
    last_angle = 0
    angle_init = 0
    # # 初始化语言播放模块
    # _already_played = False
    # last_play_time = 0   # 上一次 detect_result_read 的时间戳
    # makerobo_sensor = DistanceSensor(echo=21, trigger=20, max_distance=3, threshold_distance=0.2) #超声模块初始化
    
    while True:
        # if detect_count == 3:
        #     clbrobot.t_up(30,1)
        #     clbrobot.t_stop(0)
        #     break
        # dis = makerobo_sensor.distance * 100  # 超声模块测量距离值，并把m单位换成cm单位
        frame_counter += 1
        no_line_counter, last_angle, angle_init, _already_turn, last_turn_time= line_track(picamera_0, no_line_counter, keyflag, last_angle, angle_init, _already_turn, last_turn_time)
        #no_line_counter= line_track(picamera_0, no_line_counter, keyflag, last_angle, )
        # # --- 检测触发 ---
        # if dis <= dis_stop and not _already_played:
        #     detect_count += 1
        #     clbrobot.t_stop(3)
        #     _already_played = True
        #     last_play_time = time.time()  # 记录当前时间
        #     results = yolo_detect(picamera_1, frame_counter, model)

        #     for result in results:
        #         boxes = result.boxes  # Boxes 对象
        #         names = result.names  # 类别字典

        #     if boxes is not None:
        #         for cls in boxes.cls:
        #             name = names[int(cls)]
        #             print("识别种类：", name)
        #             if name == "仙人球":
        #                 detect_result_read('results_read/仙人掌.mp3')
        #             elif name == "薄荷叶":
        #                 detect_result_read('results_read/薄荷叶.mp3')
        #             elif name == "金枝玉叶":
        #                 detect_result_read('results_read/金枝玉叶.mp3')
        #             elif name == "吉利红":
        #                 detect_result_read('results_read/吉丽红.mp3')
        #             elif name == "绿萝":
        #                 detect_result_read('results_read/绿箩.mp3')

        # # --- 解锁条件 ---
        # elif dis > dis_stop:
        #     # 仅在播放完毕且超过result_read_delay秒后才能重新触发
        #     if _already_played and (time.time() - last_play_time >= result_read_delay):
        #         _already_played = False

        if (time.time() - last_turn_time >= turn_delay):
            _already_turn = False

def yolo_detect(picamera_1, frame_counter, model):
    # Load a pretrained YOLO11n model
    #model = YOLO("best_ncnn_model")
    frame_1 = picamera_1.capture_array()
    # YUV 转 BGR
    frame_bgr = cv2.cvtColor(frame_1, cv2.COLOR_YUV2BGR_YUYV) 
    if frame_counter >= 1:
        results = model.predict(frame_bgr)  # 只在这里用模型推理
        #print("识别结果为：", results)
        '''
        # 输出识别结果
        for result in results:
            boxes = result.boxes  # Boxes 对象
            names = result.names  # 类别字典

            if boxes is not None:
                for cls in boxes.cls:
                    print("识别种类：", names[int(cls)])
        '''
        # 绘制检测框
        annotated_frame = results[0].plot()  # 返回带框的图像
        # 显示或赋值给你原先的 widgets.Image
        image_1.value = bgr8_to_jpeg(annotated_frame)
        frame_counter = 0  # 计数器归零
    else:
        image_1.value = bgr8_to_jpeg(frame_bgr)
    return results

def detect_result_read(mp3):
    os.environ['SDL_AUDIODRIVER'] = 'alsa'
    os.environ['AUDIODEV'] = 'hw:2,0'
    pygame.mixer.init(frequency=44100, size=-16, channels=2, buffer=512)
    pygame.mixer.music.load(mp3)
    pygame.mixer.music.play()

def line_track(picamera_0, no_line_counter, keyflag, last_angle, angle_init, _already_turn, last_turn_time):
    if keyflag == 1:
        frame_0 = picamera_0.capture_array()
        hsv = cv2.cvtColor(frame_0, cv2.COLOR_BGR2HSV)

        # 检测黑色
        lower_black = np.array([0, 0, 0])
        upper_black = np.array([180, 255, 60])
        thresh1_0 = cv2.inRange(hsv, lower_black, upper_black)

        # 膨胀腐蚀
        mask_1 = cv2.erode(thresh1_0, None, iterations=2)
        mask_0 = cv2.dilate(mask_1, None, iterations=2)

        # 轮廓提取算法，其中contours_0是一个 列表（list），其中的每个元素都是一个轮廓
        contours_0, hierarchy_0 = cv2.findContours(mask_0.copy(), 1, cv2.CHAIN_APPROX_NONE)
        #print("len(contours_0): ",len(contours_0))
        if len(contours_0) > 0:
            no_line_counter = 0  # 重置丢线计数
            c = max(contours_0, key=cv2.contourArea)

            # # 角度计算
            # [vx, vy, x, y] = cv2.fitLine(c, cv2.DIST_L2, 0, 0.01, 0.01)
            # angle = math.degrees(math.atan2(vy, vx))
            # if angle < 0:
            #     angle += 180
            # print("角度为：", angle)

            # --- 角度计算 ---
            # 拟合出轮廓 c 的主方向直线，并计算这条直线的角度（即目标方向角）
            [vx, vy, x, y] = cv2.fitLine(c, cv2.DIST_L2, 0, 0.01, 0.01)
            angle = math.degrees(math.atan2(vy, vx))
            # 直线角度校准
            if angle < 0:
                angle += 180

            if angle_init == 0:
                last_angle = angle
                angle_init = 1
            else:
                # 计算前后两帧角度变化
                angle_diff = abs(angle - last_angle)
                print("last angle:", last_angle, "angle_diff:", angle_diff)
                # 角度变化超过20度（可调整阈值）
                if angle_diff > turn_angle_threshold and not _already_turn:  
                    print(f"⚠ Angle jump detected: Δ={angle_diff:.2f}° — 准备停车")
                    # 直角弯转弯操作
                    # clbrobot.t_stop(1)
                    clbrobot.t_stop(0.5)
                    clbrobot.turnLeft(30,0.4)
                    clbrobot.t_up(30,0.2)
                    _already_turn = True
                    last_turn_time = time.time()
                # 更新上一帧角度
                last_angle = angle

            M = cv2.moments(c)
            if M['m00'] != 0:
                cx = int(M['m10'] / M['m00'])
                # cy = int(M['m01'] / M['m00'])

                # cx可视化操作，在云台摄像头图像上画出 cx cx_min cx_max
                cv2.line(frame_0, (cx, 0), (cx, 120), (0, 0, 255), 2)
                cv2.line(frame_0, (cx_max, 0), (cx_max, 120), (0, 255, 255), 2)
                cv2.line(frame_0, (cx_min, 0), (cx_min, 120), (0, 255, 255), 2)

                # cv2.line(frame_0, (0, 60), (180, 60), (255, 0, 0), 2)
                # cv2.line(frame_0, (0, cy), (180, cy), (255, 255, 0), 2)

                # 轮廓检测实时可视化操作
                cv2.drawContours(frame_0, contours_0, -1, (0, 255, 0), 1)
                # 轮廓c角度实时可视化操作
                cv2.putText(
                    frame_0,                          # 要绘制的图像
                    f"Angle: {angle:.2f} deg",           # 显示的文本内容（保留两位小数）
                    (10, 20),                            # 左上角坐标 (x, y)
                    cv2.FONT_HERSHEY_SIMPLEX,            # 字体类型
                    0.5,                                 # 字体大小
                    (0, 255, 0),                         # 颜色（BGR: 绿色）
                    1,                                   # 线条粗细
                    cv2.LINE_AA)                          # 抗锯齿线
                #print("cx:",cx)
                # 巡线
                if cx >= cx_max:
                    clbrobot.turnRight(25, 0)
                    # clbrobot.turnRight(25, 0)
                    #print("➡ Turn Right!")
                elif cx < cx_max and cx > cx_min:
                    # clbrobot.t_up(30, 0)
                    clbrobot.t_up(35, 0)
                    #print("⬆ On Track!")
                elif cx <= cx_min:
                    clbrobot.turnLeft(25, 0)
                    # clbrobot.turnLeft(25, 0)
                    #print("⬅ Turn Left!")
            else:
                print("异常")
        else:
            no_line_counter += 1
            print(f"⚠ Line lost {no_line_counter} frame(s)")
            if no_line_counter >= 6:  # 连续6帧未检测到线才停车
                clbrobot.t_down(30,1.5)
                no_line_counter = 0
                print("🛑 Stop due to no line detected!")
        image.value = bgr8_to_jpeg(frame_0)
        image_thresh1_0.value = bgr8_to_jpeg(thresh1_0)
        image_mask_0.value = bgr8_to_jpeg(mask_0)
    return no_line_counter, last_angle, angle_init, _already_turn, last_turn_time
        

In [8]:
t = threading.Thread(target=Video_display)
t.setDaemon(True)
t.start()

/tmp/ipykernel_3290/3481196328.py:2: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  t.setDaemon(True)
[0:41:30.965557752] [3335]  INFO Camera camera_manager.cpp:325 libcamera v0.3.2+27-7330f29b
[0:41:30.974283620] [3336]  INFO RPI pisp.cpp:695 libpisp version v1.0.7 28196ed6edcf 29-08-2024 (16:33:32)
[0:41:30.983808808] [3336]  INFO RPI pisp.cpp:1154 Registered camera /base/axi/pcie@120000/rp1/i2c@80000/ov5647@36 to CFE device /dev/media0 and ISP device /dev/media1 using PiSP variant BCM2712_D0
[0:41:31.096692585] [3336]  WARN V4L2 v4l2_pixelformat.cpp:346 Unsupported V4L2 pixel format H264
[0:41:31.100274236] [3335]  INFO Camera camera.cpp:1197 configuring streams: (0) 180x120-RGB888 (1) 1920x1080-SGRBG16
[0:41:31.100386070] [3336]  INFO RPI pisp.cpp:1450 Sensor: /base/axi/pcie@120000/rp1/i2c@80000/ov5647@36 - Selected sensor format: 1920x1080-SGRBG10_1X10 - Selected CFE format: 1920x1080-GR16


In [9]:
stop_thread(t)
clbrobot.t_stop(0)

last angle: 90.29484991973409 angle_diff: 0.015265479416029848
last angle: 90.31011539915012 angle_diff: 0.015265479416029848
last angle: 90.29484991973409 angle_diff: 0.0
